In [1]:
import pandas as pd
import os
from config.project_config import get_data_path
from src.SolverAnalyses import ConvergenceAnalysis

In [2]:
def collect_experiment_max_times(path: str):
    """
    Durchsucht alle Unterordner des gegebenen Pfads und summiert
    pro Experiment die maximalen 'Time'-Werte aller Logdateien.

    Rückgabe:
        List[Tuple[str, float]] : (Experiment-Ordnername, Summe der max. Zeiten)
    """
    results = []

    # Nur Ordner im Hauptverzeichnis
    ordner = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]

    for experiment_ordner in ordner:
        ordnerpfad = os.path.join(path, experiment_ordner)

        # Nur Dateien in diesem Unterordner
        dateien = [
            f for f in os.listdir(ordnerpfad)
            if os.path.isfile(os.path.join(ordnerpfad, f))
        ]

        sum_max_time = 0.0

        for datei in dateien:
            logpfad = os.path.join(ordnerpfad, datei)

            try:
                df_cp_logs = ConvergenceAnalysis.parse_cp_sat_bound_log_to_dataframe(
                    file_path=logpfad
                )
            except Exception:
                continue

            if df_cp_logs is None or df_cp_logs.empty or "Time" not in df_cp_logs.columns:
                continue

            max_time = (
                df_cp_logs["Time"]
                .apply(pd.to_numeric, errors="coerce")
                .dropna()
                .max()
            )

            if pd.notna(max_time):
                sum_max_time += float(max_time)

        results.append((experiment_ordner, sum_max_time))

    return results


In [3]:
sub_directory = "2025_10_16_Bernard/logs"
path = get_data_path(
    sub_directory
)
results = collect_experiment_max_times(path)
df_results_b = pd.DataFrame(results, columns=["Experiment", "TotalMaxTime"])
df_results_b["Experiment"] = (
    df_results_b["Experiment"]
    .str.replace("Experiment_", "", regex=False)  # Präfix entfernen
    .astype(int)                                  # führende Nullen entfernen
    .apply(lambda x: f"B-{x}")                    # Präfix neu setzen
)
df_results_b

,Experiment,TotalMaxTime
0,B-1,3673.47
1,B-2,2569.68
2,B-3,1274.42
3,B-4,2615.35
4,B-5,3597.69
5,B-6,3065.76
6,B-7,2576.68
7,B-8,5149.08
8,B-9,1858.21
9,B-10,451.56


In [4]:
sub_directory = "2025_10_16_Romeo/logs"
path = get_data_path(
    sub_directory
)
results = collect_experiment_max_times(path)
df_results_r = pd.DataFrame(results, columns=["Experiment", "TotalMaxTime"])
df_results_r["Experiment"] = (
    df_results_r["Experiment"]
    .str.replace("Experiment_", "", regex=False)  # Präfix entfernen
    .astype(int)                                  # führende Nullen entfernen
    .apply(lambda x: f"R-{x}")                    # Präfix neu setzen
)
df_results_r

,Experiment,TotalMaxTime
0,R-1,4888.37
1,R-2,6231.28
2,R-3,5015.30
3,R-4,5153.81
4,R-5,4139.97
...,...,...
124,R-125,13248.18
125,R-126,15069.90
126,R-127,13902.35
127,R-128,14738.44


In [19]:
df_results = pd.concat([df_results_b, df_results_r], ignore_index=True)

In [21]:
worst_row = df_results.loc[df_results["TotalMaxTime"].idxmax()]
print("Experiment mit der längsten Gesamtzeit:")
worst_row

Experiment mit der längsten Gesamtzeit:


Experiment         R-110
TotalMaxTime    24028.91
Name: 169, dtype: object

In [22]:
median_time = df_results["TotalMaxTime"].max()
print(f"Max: {median_time/60:.0f} min")

Max: 400 min


In [16]:
median_time = df_results["TotalMaxTime"].median()
print(f"Median: {median_time/60:.0f} min")


Median: 155 min


In [17]:
mean_time = df_results["TotalMaxTime"].mean()
print(f"Mittelwert: {mean_time/60:.0f} min")


Mittelwert: 170 min


In [26]:
df_results["TotalMaxTime"].sum() /(3600*24)

np.float64(19.11403912037037)